# Example 34: Data Enrichment

Joining stream with static lookup.

## 1. Environment Setup

In [ ]:
# Install Java 11
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# Install Apache Flink
!pip install -q apache-flink

In [ ]:
from pyflink.datastream import StreamExecutionEnvironment

env = StreamExecutionEnvironment.get_execution_environment()
env.set_parallelism(1)

# Static User DB (Simulated)
user_db = {
    101: {"name": "Alice", "risk": "low"},
    102: {"name": "Bob", "risk": "high"}
}

# Stream of click events
clicks = [
    {"user_id": 101, "url": "/home"},
    {"user_id": 102, "url": "/pay"},
    {"user_id": 101, "url": "/products"}
]

def enrich(event):
    uid = event["user_id"]
    info = user_db.get(uid, {"name": "Unknown", "risk": "unknown"})
    event.update(info)
    return event

ds = env.from_collection(clicks)
ds.map(enrich).print()

env.execute("enrichment")